### 1. 电容

在电路学中，电容器在给定电压下存储电荷的能力称为电容，标记为 \( C \)。平行板电容器是一种简单的电容器，由相互平行并以空间或介电质隔离的两片导体薄板构成。

假设平行板电容器的两片极板的面积为 \( A \)，间隔距离为 \( d \)，则两片极板的面电荷分别为 \( -$\rho$, +$\rho$ \)，其中

$$
\rho = \frac{Q}{A}
$$

应用高斯定律，在两片极板之间的电场 \( E \) 为

$$
E = \frac{\rho}{\epsilon} = \frac{Q}{\epsilon A}
$$

其中，\( $\epsilon$ \) 是介质的电容率。两片极板的电势差为

$$
V = Ed = \frac{\rho d}{\epsilon} = \frac{Q d}{\epsilon A}
$$

因此，电容为

$$
C = \frac{Q}{V} = \frac{\epsilon A}{d}
$$

电容与极板面积 \( A \) 成正比，与极板间隔距离 \( d \) 成反比。这是在假设平板电容器面积 \( A \) 足够大，或间隔距离 \( d \) 远小于导板长度和宽度的情况下成立，忽略了边缘效应，即假设电场均匀分布。

（以上内容摘自[维基百科：电容](https://en.wikipedia.org/wiki/Capacitance)）

然而，在某些场景下，如 DTC 电容，边缘效应不能忽略。如果希望获得精确的电容值，不能依赖上述简单假设。



### 2. 电容问题的积分方程法

我们将极板划分为若干单元，假设不同单元的电荷可以不同。

![](Capacitor-1.png)

相比于之前假设整块极板电荷均匀分布，这样的假设考虑到了边缘效应。在给定电压下，电荷会趋于电容器的边缘。

$$
q(r) = \frac{Q_r}{S_r}
$$

其中，\( q \) 是电荷密度，\( Q \) 是单元中的总电荷，\( r \) 位于各个单元上，\( S \) 是单元的表面积。\( $q(r)$ \) 是单元上的均匀电荷密度，其位置由向量 \( r \) 表示。

在自由空间中，由点源产生的电势公式为

$$
\phi(r) = \frac{1}{\epsilon} \int_{S'} g(r, r') q(r') dS'
$$

其中，\( $S'$ \) 是电荷所在的表面面积，\( q \) 是其表面电荷密度，\( g \) 是自由空间准静态格林函数，其形式为

$$
g(r, r') = \frac{1}{4\pi} \frac{1}{|r - r'|}
$$

\( r \) 是观测点的向量，\( $r'$ \) 是点源的向量。

以上离散化过程是将电荷的积分方程转化为矩阵问题的关键部分。未知数是单元上的电荷。我们可以将积分公式应用于所有单元上的积分：

$$
\phi(r) = \frac{1}{\epsilon} \int_{S_1} g(r, r') q_1(r') dS' + \frac{1}{\epsilon} \int_{S_2} g(r, r') q_2(r') dS' + \dots
$$

其中，\( $S_l$ \) 表示单元 \( l \) 的面积。假设每个单元的电荷密度均匀，我们可以将上式改写为

$$
\phi(r) = \frac{Q_1}{\epsilon S_1} \int_{S_1} g(r, r') dS' + \frac{Q_2}{\epsilon S_2} \int_{S_2} g(r, r') dS' + \dots
$$

其中，\( $Q_i = S_i \times q_i$ \)。我们注意到对格林函数的积分可以在已知单元 \( l \) 和观测点 \( r_k \) 的位置后直接计算，即计算出系数 \( P \)：

$$
P_{pkl} = \frac{1}{\epsilon S_l} \int_{S_l} g(r_k, r') dS'
$$

构建矩阵方程 \( [P][Q] = [$\phi$] \) 进行求解。

（以上内容摘自《CIRCUIT ORIENTED ELECTROMAGNETIC MODELING USING THE PEEC TECHNIQUES》[<sup>1</sup>](#refer-anchor)）

### 3. 简单实现（以平板电容为例，附 [Matlab](Parallel_plate_capacitor.m) 代码）

实现中的难点在于系数 \( P \) 中积分的计算。对于相隔较远的单元，电荷密度产生的电势可以用中心点之间的距离近似计算。然而，对于自身单元或相邻单元，误差较大。下面推导自身电荷密度产生电势的积分值，以矩形单元为例：

$$
l_{nn} = \int_{-a}^{a} dx \int_{-b}^{b} dy \frac{1}{4\pi\epsilon \sqrt{x^2 + y^2}} = \frac{1}{4\pi\epsilon} \times 4 \times \left( \int_0^{\arctan\frac{b}{a}} d\theta \int_0^{\frac{a}{\cos\theta}} dr + \int_{\arctan\frac{b}{a}}^{\frac{\pi}{2}} d\theta \int_0^{\frac{b}{\sin\theta}} dr \right)
$$

$$
= \frac{1}{\pi\epsilon} \left[ a \ln \tan \left( \frac{x}{2} + \frac{\pi}{4} \right) \bigg|_0^{\arctan\frac{b}{a}} + b \ln \tan \left( \frac{x}{2} \right) \bigg|_{\arctan\frac{b}{a}}^{\frac{\pi}{2}} \right]
$$

对于 \( a = b \) 的特殊情况，结果为：

$$
l_{nn} = \int_{-a}^{a} dx \int_{-b}^{b} dy \frac{1}{4\pi\epsilon \sqrt{x^2 + y^2}} = \frac{2b}{\pi\epsilon} \ln(1 + \sqrt{2}) = \frac{2b}{\pi\epsilon} \times 0.8814
$$

对于相隔较远的单元，电荷密度产生的电势可以近似为：

$$
l_{mn} \approx \frac{\Delta S_n}{4 \pi \epsilon R_{mn}} = \frac{b^2}{\pi \epsilon \sqrt{(x_m - x'_n)^2 + (y_m - y'_n)^2 + (z_m - z'_n)^2}}, \quad m \neq n
$$
以平行板电容器为例，假设极板大小为\($2L_x \times 2L_y$ \)，$L_x=L_y=L=10mm$，极板间隔为\($2d$\)，我们将其等分为若干小矩形单元。

比较d从1mm到10mm分别通过数值积分计算和公式法求出的电容值，结果如下：

![](Capacitor-2.png)

取$d=4mm$绘制电荷分布，如下图所示

![](Capacitor-3.png)

计算并绘制$y=0,-3L<x<3L,-2d<z<2d$的电场分布，如下图所示

![](Capacitor-4.png)

### 4. 完整实现（以 DTC 电容为例，附 [Python](DTC_capacitor.py) 和 Fortran 代码）

同样的原理适用于其他由任意形状导电物体组成的静态电场问题。接下来我们将以台积电推出的DTC深沟槽电容为例，计算复杂结构下的电容。

![](Capacitor-DTC.png)

在深沟槽电容中，电容器的电极和介质位于硅片内部的深沟槽中。其结构包括：

    1. 沟槽壁： 沟槽的两侧通常形成电极，一个是由掺杂硅或多晶硅形成的导电电极，另一个可以是掺杂的硅衬底。
   
    2. 介电层： 在沟槽的内部表面沉积一层薄的介电材料（例如二氧化硅或氮化硅），这层材料充当电极之间的绝缘介质。
   
    3. 沟槽填充物： 沟槽内部可以填充导电材料，作为第二个电极。
   
深沟槽结构使得电容器的表面积大幅增加，即使在小的平面面积上，也可以形成长而深的电极，从而提供较大的电容量。这是深沟槽电容高电容密度的关键原因。

对于此类复杂结构，我们使用更高级的有限元网格生成器来划分表面单元，Gmsh或HFSS是通常的选择。如何使用Gmsh进行网格剖分在网络上已经有相当多的教程，我们不再过多赘述，在这里直接给出深沟槽电容在Gmsh中的.geo模型文件和生成的.msh三角网格文件。（注：我使用的Gmsh版本为4.11.1）

但在计算中还存在着与上面简单实现中一样的问题，即系数 \( P \) 中积分的计算，尤其是自身电荷密度产生电势的积分值。

参考《Calculation of CFIE impedance matrix elements with RWG and n× RWG functions》[<sup>2</sup>](#refer-anchor)一文，我们得到了该积分的闭式解，并封装成F_1_K_N函数方便调用。其余计算过程同上，最后将电容计算结果与HFSS仿真结果进行比较，误差约在4%上下。


<div id="refer-anchor"></div>

### References
[1]: Ruehli A E , Antonini G , Jiang L J .Circuit Oriented Electromagnetic Modeling Using the PEEC Techniques (Wiley - IEEE)[J].  2020.DOI:10.1002/9781119078388.

[2]: Pasi Y O, Taskinen M. Calculation of CFIE impedance matrix elements with RWG and n× RWG functions[J]. IEEE Trans. Antennas Propag, 2003, 51(8): 1837-1846.